### Import dependencies

In [1]:
import os, sys, inspect
import numpy as np
import pandas as pd
import math
import random
import sklearn
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse.linalg import svds
import matplotlib.pyplot as plt 

### Get/Insert current and project root path

In [2]:
currdir =  os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
rootdir =  os.path.dirname(currdir)
sys.path.insert(0, rootdir)

In [3]:
from rcpy.model import Popularity
from rcpy.model import ContentFilter
from rcpy.model import CollaborativeFilter
from rcpy.model.evaluator import RecallTopN

### Load data

In [4]:
articles_df = pd.read_csv(os.path.join(rootdir, "rcpy", "data", "ext", "shared_articles.csv"))
articles_df.head(5)

,timestamp,eventType,contentId,authorPersonId,authorSessionId,authorUserAgent,authorRegion,authorCountry,contentType,url,title,text,lang
0,1459192779,CONTENT REMOVED,-6451309518266745024,4340306774493623681,8940341205206233829,NaN,NaN,NaN,HTML,http://www.nytimes.com/2016/03/28/business/dea...,"Ethereum, a Virtual Currency, Enables Transact...",All of this work is still very early. The firs...,en
1,1459193988,CONTENT SHARED,-4110354420726924665,4340306774493623681,8940341205206233829,NaN,NaN,NaN,HTML,http://www.nytimes.com/2016/03/28/business/dea...,"Ethereum, a Virtual Currency, Enables Transact...",All of this work is still very early. The firs...,en
2,1459194146,CONTENT SHARED,-7292285110016212249,4340306774493623681,8940341205206233829,NaN,NaN,NaN,HTML,http://cointelegraph.com/news/bitcoin-future-w...,Bitcoin Future: When GBPcoin of Branson Wins O...,The alarm clock wakes me at 8:00 with stream o...,en
3,1459194474,CONTENT SHARED,-6151852268067518688,3891637997717104548,-1457532940883382585,NaN,NaN,NaN,HTML,https://cloudplatform.googleblog.com/2016/03/G...,Google Data Center 360° Tour,We're excited to share the Google Data Center ...,en
4,1459194497,CONTENT SHARED,2448026894306402386,4340306774493623681,8940341205206233829,NaN,NaN,NaN,HTML,https://bitcoinmagazine.com/articles/ibm-wants...,"IBM Wants to ""Evolve the Internet"" With Blockc...",The Aite Group projects the blockchain market ...,en


In [5]:
interactions_df = pd.read_csv(os.path.join(rootdir, "rcpy","data", "ext", "users_interactions.csv"))
interactions_df.head(5)

,timestamp,eventType,contentId,personId,sessionId,userAgent,userRegion,userCountry
0,1465413032,VIEW,-3499919498720038879,-8845298781299428018,1264196770339959068,NaN,NaN,NaN
1,1465412560,VIEW,8890720798209849691,-1032019229384696495,3621737643587579081,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_2...,NY,US
2,1465416190,VIEW,310515487419366995,-1130272294246983140,2631864456530402479,NaN,NaN,NaN
3,1465413895,FOLLOW,310515487419366995,344280948527967603,-3167637573980064150,NaN,NaN,NaN
4,1465412290,VIEW,-7820640624231356730,-445337111692715325,5611481178424124714,NaN,NaN,NaN


### Perfom data munging
Associate a weight/rank to each event type according to its strength.

In [6]:
event_type_keys = interactions_df.eventType.unique()
event_type_keys

array(['VIEW', 'FOLLOW', 'BOOKMARK', 'LIKE', 'COMMENT CREATED'],
      dtype=object)

In [7]:
event_type_values = [1.0, 2.0, 2.5, 3.0, 4.0]
event_type_values

[1.0, 2.0, 2.5, 3.0, 4.0]

In [8]:
event_type_dict = dict((key, value) for (key, value) in zip(event_type_keys, event_type_values))

In [9]:
interactions_df["eventStrength"] = interactions_df.eventType.apply(lambda x: event_type_dict[x])

To prevent the user-cold-start problem eliminate user with interactions less than 5.

In [10]:
interactions_count_df = pd.DataFrame(interactions_df.groupby(["personId"])["contentId"].nunique()).reset_index()
interactions_count_df.rename(columns={"contentId": "interactionsCount"}, inplace=True)

In [11]:
users_with_enough_interactions = interactions_count_df[interactions_count_df.interactionsCount >= 5]
users_with_enough_interactions.head()

,personId,interactionsCount
0,-9223121837663643404,43
1,-9212075797126931087,5
2,-9207251133131336884,7
3,-9199575329909162940,11
4,-9196668942822132778,7


In [12]:
print('# of interactions: %d' % len(interactions_df))
selected_interactions_df = interactions_df.merge(
    users_with_enough_interactions,
    how="right",
    left_on="personId",
    right_on="personId")
print('# of interactions from users with at least 5 interactions: %d' % len(selected_interactions_df))
selected_interactions_df.head()

# of interactions: 72312
# of interactions from users with at least 5 interactions: 69868


,timestamp,eventType,contentId,personId,sessionId,userAgent,userRegion,userCountry,eventStrength,interactionsCount
0,1465413032,VIEW,-3499919498720038879,-8845298781299428018,1264196770339959068,NaN,NaN,NaN,1.0,130
1,1465413046,VIEW,-3499919498720038879,-8845298781299428018,1264196770339959068,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5...,SP,BR,1.0,130
2,1464190235,VIEW,6437568358552101410,-8845298781299428018,-1157447994463607871,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_4...,SP,BR,1.0,130
3,1459429221,VIEW,-4760639635023250284,-8845298781299428018,-5149610736659242149,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_4...,SP,BR,1.0,130
4,1459274156,VIEW,-6142462826726347616,-8845298781299428018,-6283148774987755959,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_4...,SP,BR,1.0,130


Aggregate all the user interactions in a single score. (sum of weighted interactions strength smoothed by a log transformation) 

In [13]:


full_interactions_df = selected_interactions_df\
    .groupby(["personId", "contentId"])["eventStrength"]\
    .sum() \
    .apply(lambda x: math.log(1+x,2)) \
    .reset_index()
full_interactions_df.head()

,personId,contentId,eventStrength
0,-9223121837663643404,-8949113594875411859,1.000000
1,-9223121837663643404,-8377626164558006982,1.000000
2,-9223121837663643404,-8208801367848627943,1.000000
3,-9223121837663643404,-8187220755213888616,1.000000
4,-9223121837663643404,-7423191370472335463,3.169925


### Build evaluation data

In [14]:
user_column = "personId"
content_column = "contentId"
score_column = "eventStrength"
n_factors = 15

In [15]:
train_interactions_df, test_interactions_df = train_test_split(
    full_interactions_df,
    stratify=full_interactions_df[user_column],
    test_size=0.20,
    random_state=2018,)

In [16]:
print('# interactions on Train set: %d' % len(train_interactions_df))
print('# interactions on Test set: %d' % len(test_interactions_df))

# interactions on Train set: 31284
# interactions on Test set: 7822


In [17]:
full_interactions_df = full_interactions_df.set_index(user_column)
train_interactions_df = train_interactions_df.set_index(user_column)
test_interactions_df = test_interactions_df.set_index(user_column)

# full_interactions_df = full_interactions_df.reset_index()
# train_interactions_df = train_interactions_df.reset_index()
# test_interactions_df = test_interactions_df.reset_index()

In [18]:
# create user-itm score data frame
scores_df = train_interactions_df.pivot(columns=content_column, values=score_column).fillna(0)
print("user-item scores data frame: ({:d}, {:d})".format(scores_df.shape[0], scores_df.shape[1]))
scores_df.head(10)


user-item scores data frame: (1140, 2920)


contentId,-9222795471790223670,-9216926795620865886,-9194572880052200111,-9192549002213406534,-9190737901804729417,-9189659052158407108,-9184137057748005562,-9176143510534135851,-9172673334835262304,-9171475473795142532,...,9191014301634017491,9207286802575546269,9208127165664287660,9209629151177723638,9209886322932807692,9213260650272029784,9215261273565326920,9217155070834564627,9220445660318725468,9222265156747237864
personId,,,,,,,,,,,,,,,,,,,,,
-9223121837663643404,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
-9212075797126931087,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
-9207251133131336884,0.0,2.321928,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
-9199575329909162940,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
-9196668942822132778,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
-9188188261933657343,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
-9172914609055320039,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
-9156344805277471150,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
-9120685872592674274,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [19]:
# get uer-item scores matrix
scores_matrix = scores_df.values # user-item score matrix
print("user-item scores matrix: ({:d}, {:d})".format(scores_matrix.shape[0], scores_matrix.shape[1]))

user-item scores matrix: (1140, 2920)


In [20]:
# perform matrix factorization
U, sigma, Vt = svds(scores_matrix, k=n_factors) # factorized user-item score matrix
print("U: ({:d}, {:d})".format(U.shape[0], U.shape[1]))
print("sigma: ({:d})".format(len(sigma)))
print("Vt: ({:d}, {:d})".format(Vt.shape[0], Vt.shape[1]))

U: (1140, 15)
sigma: (15)
Vt: (15, 2920)


In [21]:
# reconstruct scores
scores_predicted_matrix =  np.dot(np.dot(U, np.diag(sigma)) , Vt)

In [22]:
# creat score data frame from score matrix
scores_predicted_df = pd.DataFrame(scores_predicted_matrix,
                                   columns=scores_df.columns,
                                   index=scores_df.index).transpose()
print("item-user predicted data frame: ({:d}, {:d})".format(scores_predicted_df.shape[0],
                                                            scores_predicted_df.shape[1]))
scores_predicted_df.head(10)

item-user predicted data frame: (2920, 1140)


personId,-9223121837663643404,-9212075797126931087,-9207251133131336884,-9199575329909162940,-9196668942822132778,-9188188261933657343,-9172914609055320039,-9156344805277471150,-9120685872592674274,-9109785559521267180,...,9105269044962898535,9109075639526981934,9135582630122950040,9137372837662939523,9148269800512008413,9165571805999894845,9187866633451383747,9191849144618614467,9199170757466086545,9210530975708218054
contentId,,,,,,,,,,,,,,,,,,,,,
-9222795471790223670,0.021518,-0.003161,-0.000231,0.013826,-0.000485,0.001998,-0.003025,-0.008107,0.000533,0.020652,...,0.005154,-0.017809,0.001759,0.004837,-0.025223,-0.000792,-0.001112,0.000601,-0.000384,0.010757
-9216926795620865886,0.007091,-0.000866,0.011150,0.001795,0.001422,-0.000144,0.002854,0.001714,0.000523,0.005114,...,0.002560,0.029179,-0.000012,0.000311,0.019757,0.001651,0.003856,0.011080,0.005296,0.006036
-9194572880052200111,0.003410,-0.006313,-0.014479,-0.000392,0.032582,0.001774,0.039255,-0.017318,-0.038868,0.010856,...,0.005061,0.068187,0.004929,0.032216,0.170095,0.020374,0.015971,-0.033955,0.025050,0.170980
-9192549002213406534,0.033846,0.005707,-0.002403,-0.007052,0.009163,0.003370,0.002027,0.030633,0.032550,-0.007167,...,0.012918,0.123154,0.003018,-0.016851,0.035978,0.006809,0.010932,0.012667,0.012180,0.066942
-9190737901804729417,0.027977,-0.005260,0.024479,0.006484,0.000017,0.001767,0.003577,0.005074,-0.020546,0.015774,...,0.008878,0.028745,0.000714,-0.001979,-0.047271,0.001901,0.006446,0.018271,0.005004,-0.005615
-9189659052158407108,0.024197,0.012082,0.016745,-0.000507,0.006591,0.001404,-0.003945,0.021152,0.016872,-0.002195,...,0.020020,0.073348,0.001475,0.001013,0.074214,0.007075,0.009171,0.024659,0.011652,0.045429
-9184137057748005562,0.001848,-0.000137,0.002080,-0.000504,0.000035,0.000268,0.000491,-0.000272,0.000846,0.002908,...,0.000786,0.004843,-0.000041,-0.000466,-0.002640,0.000033,0.000751,0.003158,0.001435,0.000861
-9176143510534135851,0.016298,0.004159,0.010104,0.013053,-0.003148,-0.000335,0.002440,-0.006747,0.088856,0.034673,...,-0.003214,0.107760,-0.002663,-0.007217,0.040681,-0.003551,-0.000804,0.058463,0.013055,0.044837
-9172673334835262304,0.003958,0.000920,0.000642,-0.000793,0.000630,-0.000282,0.001058,0.002689,-0.000550,0.001811,...,0.003283,0.011300,-0.000416,-0.003029,0.000951,0.000498,0.003424,0.001190,0.005132,0.000694


In [23]:
user_id = -9223121837663643404
items_to_ignore =[]
user_scores_df = scores_predicted_df[user_id].sort_values(ascending=False).reset_index().rename(columns={user_id: score_column}).head(5)

In [24]:
user_scores_df[~user_scores_df[content_column].isin(items_to_ignore)]

,contentId,eventStrength
0,2857117417189640073,0.247925
1,-1633984990770981161,0.241725
2,943818026930898372,0.236172
3,-4029704725707465084,0.228223
4,-8208801367848627943,0.208785


In [25]:
user_scores_df[~user_scores_df[content_column].isin(items_to_ignore)].sort_values(score_column, ascending=False)

,contentId,eventStrength
0,2857117417189640073,0.247925
1,-1633984990770981161,0.241725
2,943818026930898372,0.236172
3,-4029704725707465084,0.228223
4,-8208801367848627943,0.208785


In [26]:
# create model
collaborative_filter = CollaborativeFilter(
    scores_df=scores_predicted_df,
    items_df=articles_df)
collaborative_filter

In [27]:
collaborative_filter.recommend(-1479311724257856983,
                               content_column,
                               score_column,)

,contentId,eventStrength
456,-6467708104873171151,1.323902
1846,2589533162305407436,1.016670
186,-8190931845319543363,0.969033
2277,5237574390644778994,0.923988
1549,621816023396605502,0.917601
407,-6727357771678896471,0.901163
1690,1549650080907932816,0.859034
1135,-1901742495252324928,0.837682
745,-4571929941432664145,0.833693
2109,4241385837165861843,0.830376


### Preform recommendations

In [28]:
model_evaluator = RecallTopN(
    full_interactions_df=full_interactions_df,
    test_interactions_df=test_interactions_df,
    train_interactions_df=train_interactions_df,
    items_df=articles_df,
    content_column=content_column,
    score_column=score_column,
    sample_size=100)

overall_results_df, detailed_results_df = model_evaluator.evaluate(collaborative_filter)

user:        4313045637915476309 	recall@5:  0.172 	recall@10:  0.466
user:         692689608292948411 	recall@5:  0.072 	recall@10:  0.217
user:        8676130229735483748 	recall@5:  0.240 	recall@10:  0.320
user:       -9009798162809551896 	recall@5:  0.155 	recall@10:  0.259
user:       -4243635261966794110 	recall@5:  0.143 	recall@10:  0.143
user:        1915826476552544131 	recall@5:  1.000 	recall@10:  1.000
user:       -4312494396888667550 	recall@5:  0.053 	recall@10:  0.158
user:        5030175185311671155 	recall@5:  0.217 	recall@10:  0.304
user:       -3390049372067052505 	recall@5:  0.070 	recall@10:  0.116
user:        1374824663945909617 	recall@5:  0.167 	recall@10:  0.167
user:       -5052218473217969334 	recall@5:  0.600 	recall@10:  0.800
user:        7774613525190730745 	recall@5:  0.385 	recall@10:  0.692
user:       -6273276101162180437 	recall@5:  0.000 	recall@10:  0.000
user:        7459660693589256038 	recall@5:  0.000 	recall@10:  0.500
user:       -3954277

user:       -3500661007957156229 	recall@5:  0.235 	recall@10:  0.471
user:        1539552466471655556 	recall@5:  0.333 	recall@10:  0.667
user:       -6105296070998675410 	recall@5:  0.500 	recall@10:  0.667
user:       -7656701953949775820 	recall@5:  0.500 	recall@10:  0.875
user:        7948079555216525045 	recall@5:  0.270 	recall@10:  0.459
user:       -2385757031765501802 	recall@5:  0.600 	recall@10:  0.800
user:       -2979881261169775358 	recall@5:  0.148 	recall@10:  0.318
user:       -8672331451814079632 	recall@5:  0.500 	recall@10:  0.500
user:       -1292787032977259947 	recall@5:  0.308 	recall@10:  0.462
user:       -1863544689523892600 	recall@5:  0.278 	recall@10:  0.500
user:       -6944500707172804068 	recall@5:  0.115 	recall@10:  0.365
user:       -3643155458357242906 	recall@5:  0.000 	recall@10:  0.167
user:        1493119272824026132 	recall@5:  0.471 	recall@10:  0.471
user:        2682799999279860984 	recall@5:  0.286 	recall@10:  0.571
user:        4063400

user:        3094513233385472738 	recall@5:  0.250 	recall@10:  0.375
user:        3290087150125937633 	recall@5:  0.143 	recall@10:  0.143
user:       -8845298781299428018 	recall@5:  0.308 	recall@10:  0.500
user:       -5969793184455185931 	recall@5:  1.000 	recall@10:  1.000
user:        6971525809430309144 	recall@5:  0.467 	recall@10:  0.600
user:        6120111409505648242 	recall@5:  0.375 	recall@10:  0.625
user:        1908339160857512799 	recall@5:  0.524 	recall@10:  0.667
user:        8243474948679935093 	recall@5:  0.000 	recall@10:  0.000
user:       -4293690759387208926 	recall@5:  0.000 	recall@10:  0.000
user:       -5795907895499274955 	recall@5:  0.000 	recall@10:  0.000
user:        7302126548210559734 	recall@5:  0.333 	recall@10:  0.500
user:       -4951114616026800382 	recall@5:  0.400 	recall@10:  0.600
user:       -4167415844282118439 	recall@5:  0.000 	recall@10:  0.000
user:        4254153380739593270 	recall@5:  0.361 	recall@10:  0.500
user:       -8560472

user:        8872819156169667456 	recall@5:  0.400 	recall@10:  0.500
user:        4335864930735272674 	recall@5:  0.000 	recall@10:  0.000
user:       -7531858294361854119 	recall@5:  0.375 	recall@10:  0.375
user:       -6067316262393890508 	recall@5:  0.353 	recall@10:  0.588
user:        6018247344671480458 	recall@5:  0.000 	recall@10:  0.111
user:        8766802480854827422 	recall@5:  0.273 	recall@10:  0.455
user:       -6551899843081273989 	recall@5:  0.000 	recall@10:  0.000
user:        4432373331608915019 	recall@5:  0.250 	recall@10:  0.250
user:       -8251340065348651782 	recall@5:  0.667 	recall@10:  0.667
user:        6970482293998499949 	recall@5:  0.000 	recall@10:  0.000
user:       -6805567438224540711 	recall@5:  0.667 	recall@10:  0.667
user:        6013800624570706808 	recall@5:  0.000 	recall@10:  0.000
user:       -1799631734242668035 	recall@5:  0.375 	recall@10:  0.500
user:         627408625947732191 	recall@5:  0.286 	recall@10:  0.571
user:        -634318

user:        5853826697811091550 	recall@5:  0.500 	recall@10:  0.875
user:        3586672003786969120 	recall@5:  0.250 	recall@10:  0.583
user:        5598109800222509376 	recall@5:  0.000 	recall@10:  0.000
user:        7421498048184238966 	recall@5:  0.667 	recall@10:  0.667
user:       -4733247784100640601 	recall@5:  0.000 	recall@10:  0.000
user:       -5823562314929634489 	recall@5:  0.000 	recall@10:  0.286
user:        5328649570833345873 	recall@5:  0.500 	recall@10:  0.833
user:        8341545449730058951 	recall@5:  0.200 	recall@10:  0.400
user:        8239286975497580612 	recall@5:  0.333 	recall@10:  0.533
user:       -7041401352840181657 	recall@5:  0.375 	recall@10:  0.375
user:        4965319960651517505 	recall@5:  0.500 	recall@10:  1.000
user:       -7482701687530266291 	recall@5:  0.500 	recall@10:  0.500
user:         899165159624701221 	recall@5:  0.000 	recall@10:  0.000
user:        7696404421947461044 	recall@5:  0.667 	recall@10:  0.778
user:       -5292464

user:       -4465926797008424436 	recall@5:  0.545 	recall@10:  0.727
user:       -7619256453002568786 	recall@5:  0.500 	recall@10:  1.000
user:       -8781635134606732409 	recall@5:  0.000 	recall@10:  0.000
user:       -6734098108417569342 	recall@5:  0.500 	recall@10:  0.500
user:        6646901263143710184 	recall@5:  0.000 	recall@10:  0.000
user:       -4125205337625989832 	recall@5:  0.429 	recall@10:  0.571
user:        7001798922437039535 	recall@5:  0.000 	recall@10:  0.000
user:        1070632260016389736 	recall@5:  0.000 	recall@10:  0.200
user:        2111379813881770134 	recall@5:  0.500 	recall@10:  0.500
user:        -387586131996064525 	recall@5:  0.444 	recall@10:  0.667
user:        5220802546024427652 	recall@5:  0.200 	recall@10:  0.200
user:       -7253045327330314894 	recall@5:  0.000 	recall@10:  0.000
user:       -3011221777935346594 	recall@5:  0.000 	recall@10:  0.000
user:        3856314584219831841 	recall@5:  0.333 	recall@10:  0.333
user:        1072886

user:       -2050699458865052139 	recall@5:  0.083 	recall@10:  0.167
user:       -8207027873334509187 	recall@5:  0.000 	recall@10:  0.000
user:        1377941589660984019 	recall@5:  0.750 	recall@10:  0.750
user:       -1561982036714087726 	recall@5:  0.500 	recall@10:  0.500
user:       -2772844562500836582 	recall@5:  0.125 	recall@10:  0.375
user:       -5349775063975483956 	recall@5:  0.500 	recall@10:  1.000
user:        6675870946781601732 	recall@5:  0.500 	recall@10:  1.000
user:       -4355190001103094206 	recall@5:  0.500 	recall@10:  0.500
user:       -6120788038252469111 	recall@5:  0.000 	recall@10:  0.000
user:       -8699750646678621887 	recall@5:  0.667 	recall@10:  1.000
user:        7530102388348371118 	recall@5:  0.400 	recall@10:  0.600
user:        3743262342828860520 	recall@5:  0.750 	recall@10:  0.750
user:       -6196110550195246902 	recall@5:  0.500 	recall@10:  0.500
user:        1059826223086140014 	recall@5:  0.500 	recall@10:  0.500
user:       -2376088

user:        -663224541983667655 	recall@5:  0.667 	recall@10:  0.667
user:        8676384235153995185 	recall@5:  0.000 	recall@10:  0.000
user:        2736634093130347762 	recall@5:  0.000 	recall@10:  0.000
user:        3763760030606782798 	recall@5:  0.000 	recall@10:  0.000
user:       -6361883726252419530 	recall@5:  0.250 	recall@10:  0.500
user:       -5676352709327073303 	recall@5:  0.500 	recall@10:  0.500
user:        4297386640890689301 	recall@5:  0.600 	recall@10:  1.000
user:       -6276733705223916438 	recall@5:  0.500 	recall@10:  0.500
user:       -3626028832820976226 	recall@5:  0.000 	recall@10:  0.000
user:         781132119141619887 	recall@5:  0.500 	recall@10:  0.500
user:        7703284633155187943 	recall@5:  0.167 	recall@10:  0.500
user:       -9196668942822132778 	recall@5:  0.000 	recall@10:  0.000
user:       -4448388750105189364 	recall@5:  0.333 	recall@10:  0.667
user:        -174458633445209100 	recall@5:  0.000 	recall@10:  0.250
user:        6967147

user:        7741213197781008422 	recall@5:  0.667 	recall@10:  1.000
user:       -1007001694607905623 	recall@5:  0.000 	recall@10:  0.000
user:        7215563157885540458 	recall@5:  0.000 	recall@10:  0.000
user:        7705707768530115638 	recall@5:  0.000 	recall@10:  0.000
user:       -6085280990332832124 	recall@5:  0.000 	recall@10:  0.000
user:       -6246099423021213661 	recall@5:  0.000 	recall@10:  0.000
user:       -1160159014793528221 	recall@5:  0.000 	recall@10:  0.000
user:       -3815514359891076081 	recall@5:  0.500 	recall@10:  1.000
user:        -900146660487611772 	recall@5:  0.000 	recall@10:  0.000
user:        1693920335354805642 	recall@5:  0.500 	recall@10:  0.500
user:        2405211284771043881 	recall@5:  1.000 	recall@10:  1.000
user:        6090234987314110812 	recall@5:  0.000 	recall@10:  0.000
user:       -2108111265827636778 	recall@5:  0.000 	recall@10:  0.000
user:        6590718530693270591 	recall@5:  0.000 	recall@10:  0.000
user:       -7847088

In [29]:
overall_results_df.head(10)

,model_name,recall_top5,recall_top10
0,collaborative-filter,0.26336,0.393761


In [30]:
detailed_results_df.head(10)

,nm_hits_top10,nm_hits_top5,nm_interacted_items,recall_top10,recall_top5,user_id
44,0,0,192,0.000000,0.000000,3609194402293569455
63,0,0,134,0.000000,0.000000,-2626634673110551643
70,0,0,130,0.000000,0.000000,-1032019229384696495
34,0,0,117,0.000000,0.000000,-1443636648652872475
134,28,13,88,0.318182,0.147727,-2979881261169775358
150,6,1,80,0.075000,0.012500,-3596626804281480007
37,17,9,73,0.232877,0.123288,1116121227607581999
95,15,4,69,0.217391,0.057971,-9016528795238256703
1,15,5,69,0.217391,0.072464,692689608292948411
283,9,3,68,0.132353,0.044118,3636910968448833585
